In [1]:
import pandas as pd

#Cleaning players_data
   
players_data = pd.read_csv("Dataset/players.csv")
#players_data.dropna(thresh=2, inplace=True)
players_data = players_data[players_data.pos.notnull()]
players_data.rename(columns={"weight": "w"}, inplace=True)
players_data.rename(columns={"height": "weight"}, inplace=True)
players_data.rename(columns={"w": "height"}, inplace=True)
players_data.drop(players_data[players_data['weight'] < 40].index, inplace = True)
players_data.drop(players_data[players_data['height'] < 140].index, inplace = True)
players_data.drop(columns=["firstseason","lastseason","deathDate"],inplace=True)
players_data.reset_index()
players_data.describe()

,weight,height
count,682.000000,682.000000
mean,72.665689,172.476540
std,3.137050,21.279833
min,63.000000,140.000000
25%,70.000000,157.250000
50%,73.000000,170.000000
75%,75.000000,185.000000
max,80.000000,254.000000


In [2]:
teams_data = pd.read_csv("Dataset/teams.csv")


In [3]:
players_teams_data = pd.read_csv("Dataset/players_teams.csv")
awards_players_data = pd.read_csv("Dataset/awards_players.csv")
players_teams_data['awards'] = 0
for idx, player in players_teams_data.iterrows():
    awards_until_target_season = awards_players_data[(awards_players_data['playerID'] == player['playerID']) & (awards_players_data['year'] < player['year'])]
    if(not awards_until_target_season.empty):
        players_teams_data.loc[[idx], 'awards'] = len(awards_until_target_season)

for idx, team in teams_data.iterrows():
        team_awards_until_target_season = players_teams_data[(players_teams_data["year"] == team["year"] )&( players_teams_data["tmID"] == team["tmID"])]
        team_awards_num = team_awards_until_target_season["awards"].sum()
        teams_data.loc[[idx], 'awards'] = team_awards_num
teams_data        

,year,tmID,confID,rank,playoff,firstRound,semis,finals,name,o_fgm,...,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena,awards
0,1,CHA,EA,8,N,NaN,NaN,NaN,Charlotte Sting,812,...,5,11,3,13,5,16,6475,90963,Charlotte Coliseum,0.0
1,2,CHA,EA,4,Y,W,W,L,Charlotte Sting,746,...,11,5,7,9,15,6,6500,105525,Charlotte Coliseum,0.0
2,3,CHA,EA,2,Y,L,NaN,NaN,Charlotte Sting,770,...,11,5,7,9,12,9,6450,106670,Charlotte Coliseum,0.0
3,4,CHA,EA,2,Y,L,NaN,NaN,Charlotte Sting,787,...,13,4,5,12,12,12,6850,120061,Charlotte Coliseum,0.0
4,5,CHA,EA,5,N,NaN,NaN,NaN,Charlotte Sting,745,...,10,7,6,11,8,12,6900,116383,Charlotte Coliseum,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,4,WAS,EA,7,N,NaN,NaN,NaN,Washington Mystics,896,...,3,14,6,11,7,17,6875,238710,Verizon Center,1.0
111,5,WAS,EA,4,Y,L,NaN,NaN,Washington Mystics,873,...,11,6,6,11,9,11,6850,214448,Verizon Center,1.0
112,6,WAS,EA,5,N,NaN,NaN,NaN,Washington Mystics,847,...,10,7,6,11,9,11,6900,171501,Verizon Center,1.0
113,7,WAS,EA,4,Y,L,NaN,NaN,Washington Mystics,1016,...,13,4,5,12,12,8,6850,133255,Verizon Center,2.0
